# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
# from api_keys import geoapify_key
from dotenv import load_dotenv
import os

load_dotenv()
geoapify_key = os.getenv("geoapify_key")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Capitalize the City Names
city_data_df["City"] = city_data_df["City"].str.title()


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Burang,30.2500,81.1667,12.65,75,73,1.99,CN,1722360916
1,1,Qaqortoq,60.7167,-46.0333,10.40,44,0,14.95,GL,1722360917
2,2,Grytviken,-54.2811,-36.5092,-3.19,65,23,3.29,GS,1722360918
3,3,Puerto Ayora,-0.7393,-90.3518,25.98,94,98,3.13,EC,1722360919
4,4,Bredasdorp,-34.5322,20.0403,7.91,72,20,1.26,ZA,1722360920


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Use stipulations for 'ideal weather condition'
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,Aligudarz,33.4006,49.6949,26.14,17,0,1.04,IR,1722360977
70,70,Yeraliyev,43.2000,51.6500,25.49,54,0,1.65,KZ,1722360997
76,76,Turkmenbasy,40.0222,52.9552,26.43,61,0,3.09,TM,1722361004
91,91,Stanley,54.8680,-1.6985,21.01,49,0,4.12,GB,1722361022
101,101,The Pas,53.8251,-101.2541,26.01,57,0,2.57,CA,1722361034
134,134,Tralee,52.2704,-9.7026,21.14,64,0,3.09,IE,1722361073
353,353,Nova Vicosa,-17.8919,-39.3719,26.45,65,0,4.22,BR,1722361329
404,404,Beira,-19.8436,34.8389,22.05,100,0,2.68,MZ,1722361465
410,410,Fort Mcmurray,56.7268,-111.3810,22.73,53,0,2.06,CA,1722361470
544,544,Yuxia,34.0615,108.6291,24.34,69,0,2.59,CN,1722361506


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,Aligudarz,IR,33.4006,49.6949,17,
70,Yeraliyev,KZ,43.2000,51.6500,54,
76,Turkmenbasy,TM,40.0222,52.9552,61,
91,Stanley,GB,54.8680,-1.6985,49,
101,The Pas,CA,53.8251,-101.2541,57,
134,Tralee,IE,52.2704,-9.7026,64,
353,Nova Vicosa,BR,-17.8919,-39.3719,65,
404,Beira,MZ,-19.8436,34.8389,100,
410,Fort Mcmurray,CA,56.7268,-111.3810,53,
544,Yuxia,CN,34.0615,108.6291,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # 10,000 metres
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Aligudarz - nearest hotel: هتل جهانگردی الیگودرز
Yeraliyev - nearest hotel: Kuryk
Turkmenbasy - nearest hotel: Hazar Myhmanhanasy
Stanley - nearest hotel: Hotel 52
The Pas - nearest hotel: Wescana Inn
Tralee - nearest hotel: The Ashe Hotel
Nova Vicosa - nearest hotel: Pousada Pontal da Barra
Beira - nearest hotel: Hotel Savoy
Fort Mcmurray - nearest hotel: Nomad Hotel and Suites
Yuxia - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
52,Aligudarz,IR,33.4006,49.6949,17,هتل جهانگردی الیگودرز
70,Yeraliyev,KZ,43.2000,51.6500,54,Kuryk
76,Turkmenbasy,TM,40.0222,52.9552,61,Hazar Myhmanhanasy
91,Stanley,GB,54.8680,-1.6985,49,Hotel 52
101,The Pas,CA,53.8251,-101.2541,57,Wescana Inn
134,Tralee,IE,52.2704,-9.7026,64,The Ashe Hotel
353,Nova Vicosa,BR,-17.8919,-39.3719,65,Pousada Pontal da Barra
404,Beira,MZ,-19.8436,34.8389,100,Hotel Savoy
410,Fort Mcmurray,CA,56.7268,-111.3810,53,Nomad Hotel and Suites
544,Yuxia,CN,34.0615,108.6291,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)


# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)